In [1]:
import slicer

# Load the 3D TIFF file
file_name = ''
tif_file_path = ''
volume_node = slicer.util.loadVolume(tif_file_path+file_name)

In [2]:
# Segment outer and inner ranges using segment editor
segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
segmentationNode.CreateDefaultDisplayNodes()  # Only needed for display
segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(volume_node)
addedSegmentID = segmentationNode.GetSegmentation().AddEmptySegment("outer range")

segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()
segmentEditorWidget.setMRMLScene(slicer.mrmlScene)
segmentEditorNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentEditorNode")
segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)
segmentEditorWidget.setSegmentationNode(segmentationNode)
segmentEditorWidget.setMasterVolumeNode(volume_node)

segmentEditorWidget.setActiveEffectByName("Threshold")
effect = segmentEditorWidget.activeEffect()
effect.setParameter("MinimumThreshold", "150")
effect.setParameter("MaximumThreshold", "1000")
effect.self().onApply()

In [3]:
wrap_solidify_module = slicer.util.getModule('SegmentEditorWrapSolidify')
# IMPORTANT: check if "surface wrapping solidy" module has been installed, download it in the extension!!!

In [5]:
# Apply the Wrap Solidify effect
segmentEditorWidget.setActiveEffectByName('Wrap Solidify')
wrap_solidify_effect = segmentEditorWidget.activeEffect()
# wrap_solidify_effect.setParameter("InputSegmentation", segmentationNode.GetID())
wrap_solidify_effect.setParameter("region", "largestCavity")
wrap_solidify_effect.setParameter("smoothingFactor", 1.0)
wrap_solidify_effect.setParameter("shrinkwrapIterations", 1.0)
wrap_solidify_effect.setParameter("remeshOversampling", 1.0)
# wrap_solidify_effect.setParameter("OutputSegmentation", segmentationNode.GetID())
wrap_solidify_effect.self().onApply()

Next step: Manually trim the mask using the scissors tool in the Segment Editor to confine it to the lesion region. Then, export the mask for storage.